In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa

## Topic Seeds

In [3]:
from casa import Cadet
import numpy as np

In [50]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20")
cadet

[INFO] 2021-01-27 23:00:32,986 gensim.utils: loading Word2VecKeyedVectors object from E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-01-27 23:00:33,956 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-01-27 23:00:33,956 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-01-27 23:00:33,957 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-01-27 23:00:33,957 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-01-27 23:00:33,958 gensim.utils: loaded E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-01-27 23:00:33,958 gensim.models.keyedvectors: precomputing L2-norms of word weight vectors
[INFO] 2021-01-27 23:00:33,968 gensim.models.keyedvectors: precomputing L2-norms of ngram weight vectors


<Cadet: FastText(9851, 200), Seeds(103)>

In [5]:
text = "我昨天跟遠傳分手了，他送的iphone該還回去嗎"
cadet.detect(text, verbose=True)

tokens:  ['我', '昨天', '遠傳', '分', '手', '了', '他', '送', 'iphone', '該', '還', '回去', '嗎']
ent_scores [0.6359725  0.6595124  1.         0.56385303 0.55743235 0.43274459]
srv_scores [0.58001447 0.47847968 0.48819646 1.        ]
seed_scores(topn) [1.         0.58001447 0.56701541 0.54494095 0.53985411]


{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '其他電信'],
 'entity_probs': array([0.01200945, 0.01229551, 0.94361807, 0.01117383, 0.01110232,
        0.00980082]),
 'service': ['資費方案', '通訊品質', '加值服務', '其他'],
 'service_probs': array([0.01163998, 0.01051614, 0.01061882, 0.96722507]),
 'seeds': ['手機', '通話費', '轉4G', '電銷', '方案'],
 'seed_probs': array([0.52720503, 0.0063446 , 0.00626266, 0.00612593, 0.00609485])}

## Thread-level

In [52]:
import pickle
with open(casa.get_data_path() / "threads/cht-2020-merged.pkl", "rb") as fin:
    threads = pickle.load(fin)

In [53]:
from casa.cadet import CadetProcessor
from tqdm.auto import tqdm

In [54]:
threads[0].main.post_type

'主文'

In [55]:
cadet_proc = CadetProcessor(cadet)
for thread_x in tqdm(threads):
    thread_x.process(cadet_proc)

In [56]:
with open(casa.get_data_path() / "threads/cht-2020-merged-cadet-op20.pkl", "wb") as fout:
    pickle.dump(threads, fout)

## Prioritize threads

In [59]:
with open(casa.get_data_path() / "threads/cht-2020-merged-cadet-op20.pkl", "rb") as fin:
    threads = pickle.load(fin)

In [70]:
def get_aspect(result):    
    if not result:
        return (None, None, None, None)
    
    entity_probs = result["entity_probs"]    
    if entity_probs.max() < 0.4:
        entity = None        
        ent_prob = None
    else:
        entity = result["entity"][np.argmax(entity_probs)]
        ent_prob = entity_probs[np.argmax(entity_probs)]
        
    service_probs = result["service_probs"]
    if service_probs.max() < 0.4:
        service = None        
        srv_prob = None
    else:
        service = result["service"][np.argmax(service_probs)]
        srv_prob = service_probs[np.argmax(service_probs)]
    return (entity, service, ent_prob, srv_prob)

In [62]:
print(threads[1].main.title)
threads[1].print_thread()
threads[1].main.cadet_title

#請益 種花續約vs台灣之星購機方案
Main: <OpinionProc [Dcard] 國立中山大學: 從有手機開始全家都是種花的忠實客戶但最近... ()>
-- <OpinionProc [Dcard] 烤鴨王: 上網查雙NP ()>
-- <OpinionProc [Dcard] 長榮大學: 每一家對老客戶的優惠都....唉 ()>
-- <OpinionProc [Dcard] 國立高雄師範大學: 直接看雙NP+1 ()>
-- <OpinionProc [Dcard] 南臺科技大學: 買11 pro空機準備從台星跳種花給你參... ()>
-- <OpinionProc [Dcard] 國立虎尾科技大學: 你先辦試用卡在決定吧！！不然你有可能網路... ()>
-- <OpinionProc [Dcard] 國立臺北教育大學: 推薦先試用再說，興高采烈換了中華的苦主在... ()>
-- <OpinionProc [Dcard] 國立臺北大學: 真的～夜間拍攝很重要 推樓上先試用！常去... ()>
-- <OpinionProc [Dcard] 國立臺北大學: 我朋友每到一段時間就會換電信才能夠把各家... ()>
-- <OpinionProc [Dcard] 中國醫藥大學: 真的推薦先去辦試用如果你常活動的地區離都... ()>
-- <OpinionProc [Dcard] 華威大學: 換11吧 先換起來 之後用不順再跳槽 現... ()>


{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '其他電信'],
 'entity_probs': array([0.48837139, 0.0062503 , 0.00578304, 0.48837139, 0.00578089,
        0.005443  ]),
 'service': ['資費方案', '通訊品質', '加值服務', '其他'],
 'service_probs': array([0.96726538, 0.01036411, 0.01098799, 0.01138252]),
 'seeds': ['方案', '資費', '綁約', '購機', '優惠'],
 'seed_probs': array([0.11303094, 0.11303094, 0.11303094, 0.11303094, 0.11303094])}

In [71]:
thread_x = threads[1]
def get_thread_aspect(thread_x):
    entity = None
    service = None
    entity_prob = None
    service_prob = None

    if thread_x.main:
        cadet_result = getattr(thread_x.main, "cadet_title", None)
        title_entity, title_service, title_ep, title_sp = get_aspect(cadet_result)    
        cadet_result = getattr(thread_x.main, "cadet_result", None)
        main_entity, main_service, main_ep, main_sp = get_aspect(cadet_result)
        # print("title: ", title_entity, title_service)
        # print("main: ", main_entity, main_service)
        entity = title_entity or main_entity
        service = title_service or main_service
        entity_prob = title_ep or main_ep
        service_prob = title_sp or main_sp

    for reply_x in thread_x.replies:
        cadet_result = getattr(reply_x, "cadet_result", None)
        reply_entity, reply_service, reply_ep, reply_sp = get_aspect(cadet_result)
        entity = entity or reply_entity
        service = service or reply_service
        entity_prob = entity_prob or reply_ep
        service_prob = service_prob or reply_sp
        # print("reply: ", reply_entity, reply_service)
        if entity and service:
            break
    return (entity, service, entity_prob, service_prob)

In [72]:
get_thread_aspect(threads[0])

('中華電信', '其他', 0.9999971555176705, 0.9999092083704556)

'D2020010117152852985906707873691962663605748902362'

In [75]:
data = []
for thread_x in tqdm(threads):
    tid = threads[0].get_opinion().id    
    entity, service, ep, sp = get_thread_aspect(thread_x)
    data.append((tid, entity, service, ep, sp))


In [78]:
import pandas as pd
thread_frame = pd.DataFrame(data, columns=["thread_id", "entity", "service", "ent_prob", "srv_prob"])

In [37]:
get_aspect([getattr(x, "cadet_result") for x in threads[0].opinions()][0])

('中華電信', '其他')

In [87]:
thread_frame.groupby("entity") \
    .aggregate(n=("entity", len), avg=("ent_prob", "mean"), sd=("ent_prob", "std"))\
    .sort_values("n", ascending=False)

,n,avg,sd
entity,,,
中華電信,16123,0.815552,0.217922
台灣之星,8921,0.877564,0.176779
遠傳電信,7529,0.885528,0.170036
台灣大哥大,7481,0.848777,0.194544
亞太電信,4744,0.949130,0.059831
其他電信,721,0.603363,0.158857


In [88]:
thread_frame.groupby("service") \
    .aggregate(n=("service", len), avg=("srv_prob", "mean"), sd=("srv_prob", "std"))\
    .sort_values("n", ascending=False)

,n,avg,sd
service,,,
資費方案,27362,0.806484,0.234768
通訊品質,14322,0.819098,0.229374
加值服務,7514,0.899231,0.186313
其他,2776,0.908723,0.163626


In [95]:
thread_frame.groupby(["entity", "service"]).aggregate(n=("thread_id", len)).sort_values(["entity", "n"], ascending=False)

n
entity service      
遠傳電信   資費方案     3500
       通訊品質     2206
       加值服務      831
       其他        392
台灣大哥大  資費方案     3963
       通訊品質     1664
       加值服務      872
       其他        448
台灣之星   資費方案     5595
       通訊品質     1979
       加值服務      599
       其他        331
其他電信   資費方案      454
       加值服務      105
       通訊品質       98
       其他         44
亞太電信   資費方案     2592
       通訊品質     1155
       加值服務      515
       其他        146
中華電信   資費方案     7090
       通訊品質     4329
       加值服務     3031
       其他        818